In [7]:
import numpy as np
import gvar as gv
import re 
import pandas as pd 
import sys
import copy
import tables as h5
import h5py
import os 
import time
import re
sys.path.insert(0, '/home/gbradley/nucleon_elastic_FF')
from nucleon_elastic_ff.data.h5io import get_dsets 


In [93]:
h5fname = '/home/gbradley/Downloads/example_data.hdf5'
def read_example_data(h5fname):
    """Read the example data"""
    data = {}
    with h5py.File(h5fname, 'r') as ifile:
        dset = ifile['data']
        for key in dset.keys():
            data[key] = ifile['data'][key][:]
    for key in ['13','14','15','16']:
        data[int(key)] = data.pop(key)

    return data


In [96]:
data=read_example_data(h5fname)
ds = {key: gv.dataset.avg_data(val) for key, val in data.items()}


{'heavy-light': array([0.2505770(22), 0.0503621(20), 0.01168217(76), 0.00348845(59),
       0.00096940(20), 0.00042637(15), 0.000148455(53), 0.000071900(37),
       0.000027770(14), 0.0000136105(93), 5.5628(38)e-06, 2.7053(24)e-06,
       1.1448(10)e-06, 5.5082(60)e-07, 2.3831(28)e-07, 1.1352(16)e-07,
       4.9867(75)e-08, 2.3548(42)e-08, 1.0461(21)e-08, 4.901(11)e-09,
       2.1974(56)e-09, 1.0228(31)e-09, 4.621(15)e-10, 2.1407(85)e-10,
       9.696(42)e-11, 4.485(23)e-11, 2.033(12)e-11, 9.407(64)e-12,
       4.259(33)e-12, 1.969(18)e-12, 8.915(90)e-13, 4.134(49)e-13,
       1.876(25)e-13, 8.73(14)e-14, 3.938(71)e-14, 1.849(38)e-14,
       8.31(20)e-15, 3.94(10)e-15, 1.737(56)e-15, 8.26(29)e-16,
       3.52(15)e-16, 1.725(84)e-16, 7.10(43)e-17, 3.74(24)e-17,
       1.44(13)e-17, 8.36(69)e-18, 2.99(36)e-18, 2.25(21)e-18,
       1.44(15)e-18, 2.06(20)e-18, 3.63(37)e-18, 7.79(70)e-18,
       1.60(13)e-17, 3.54(25)e-17, 7.63(46)e-17, 1.686(87)e-16,
       3.69(16)e-16, 8.19(31)e-16, 1.80

In [154]:
# h5fname = '/home/gbradley/c51_corr_analysis/tests/data/a09m135_s_avg_srcs0-15.h5'
h5fname = '/home/gbradley/c51_corr_analysis/tests/data/C13/C13-b_4002.ama.h5'


In [ ]:
patterns = [
    "3pt",
    "_tsep(?P<tsep>[0-9]|[0-9]+)",  # must match `_tsep` and stores the following numbers (any length)
    "/NUCL_(?P<quark>U|D)",  # Store U or D in quark
    "_MIXED_NONREL",  # Not sure if this changes. Not stored for now
    "_l(?P<l>[0-9]+)",  # action parameters?
    "_g(?P<g>[0-15]+)",
    "/src(?P<src>[0-9\.]+)",  # Stores numbers + . to store decimals. Must escape .
    "_snk(?P<snk>[0-9\.]+)",  # Stores numbers + . to store decimals. Must escape .
    "/qz(?P<qz>[\+\-0-9]+)", 
    "_qy(?P<qy>[\+\-0-9]+)", 
    "_qx(?P<qx>[\+\-0-9]+)", 
    
]

In [155]:

pattern = "(?P<parity>proton|proton\_np)"
pattern += "_(?P<isospin>DD|UU)"
pattern += "_(?P<spin>dn_dn|up_up)"
pattern += "_tsep_[\-]*(?P<tsep>[0-9]+)"
pattern += ".*(?P<current>A3|V4).*cfgs\_srcs"


In [156]:
# columns = ["nucleon", "current", "tsep", "cfg", "t", "isospin", "parity", "spin", "corr"]
columns = ["tsep", "quark", "l", "g", "src", "snk","qz","qy","qx"]


In [158]:
data_frames = []

with h5py.File(h5fname, "r") as h5f:
    dsets = get_dsets(h5f)
    # print(dsets)

    for key, dset in dsets.items():
        match = re.search(pattern, key)
        if match:
            info = match.groupdict()

            nucleon_parity = info.pop("parity").split("_")
            info["nucleon"] = nucleon_parity[0]
            info["parity"] = -1 if len(nucleon_parity) == 2 else 1
            
            isospin = info.pop("isospin")
            info["isospin"] = 1 if isospin == "UU" else -1            

            current_key = key.replace("cfgs_srcs", "local_curr")
            curr_dset = h5f[current_key]

            cfgs = dset[:, 0]
            corr = (
                curr_dset[()].real if info["current"] in ["V4"] else curr_dset[()].imag
            )
            ts = range(corr.shape[-1])

            tmp_df = (
                pd.DataFrame(index=cfgs, columns=ts, data=corr)
                .unstack()
                .reset_index()
                .rename(columns={"level_0": "t", "level_1": "cfg", 0: "corr"})
            )
            for key, val in info.items():
                tmp_df[key] = val
            data_frames.append(tmp_df.astype({"tsep": int}))



df = pd.concat(
    data_frames, 
    ignore_index=True, 
).reindex(columns, axis=1).sort_values(columns).reset_index(drop=True)
df.head()

[2022-08-16 19:33:27,705|lqcd correlator analysis@INFO] Locating all dsets of h5 file `/home/gbradley/c51_corr_analysis/tests/data/a09m135_s_avg_srcs0-15.h5`


,nucleon,current,tsep,cfg,t,isospin,parity,spin,corr
0,proton,A3,3,78.0,0,-1,-1,dn_dn,-1.349853e-09
1,proton,A3,3,78.0,0,-1,-1,dn_dn,-1.064086e-09
2,proton,A3,3,78.0,0,-1,-1,up_up,2.566949e-09
3,proton,A3,3,78.0,0,-1,-1,up_up,2.696527e-09
4,proton,A3,3,78.0,0,-1,1,dn_dn,1.716125e-09


In [159]:
spin_avg_df = df.groupby(
    ["nucleon", "current", "tsep", "cfg", "t", "isospin", "parity"], as_index=False
)["corr"].mean()

spin_avg_df.head()



,nucleon,current,tsep,cfg,t,isospin,parity,corr
0,proton,A3,3,78.0,0,-1,-1,7.123843e-10
1,proton,A3,3,78.0,0,-1,1,-5.496932e-10
2,proton,A3,3,78.0,0,1,-1,1.118381e-09
3,proton,A3,3,78.0,0,1,1,-8.362174e-10
4,proton,A3,3,78.0,1,-1,-1,9.552208e-10


In [160]:
tmp = spin_avg_df.copy()
tmp["corr"] *= tmp["parity"]
spin_parity_avg_df = tmp.groupby(
    ["nucleon", "current", "tsep", "cfg", "t", "isospin",  ], as_index=False
)[["corr"]].mean()

spin_parity_avg_df.head()

,nucleon,current,tsep,cfg,t,isospin,corr
0,proton,A3,3,78.0,0,-1,-6.310388e-10
1,proton,A3,3,78.0,0,1,-9.772991e-10
2,proton,A3,3,78.0,1,-1,-6.853185e-10
3,proton,A3,3,78.0,1,1,-5.990430e-10
4,proton,A3,3,78.0,2,-1,-6.418973e-10


In [161]:
tmp = spin_parity_avg_df.copy()
tmp["corr"] *= tmp["isospin"]
isospin_spin_parity_avg_df = (
    tmp.groupby(["nucleon", "current", "tsep", "cfg",  "t"], as_index=False)["corr"]
    .sum()
)
isospin_spin_parity_avg_df.head()

,nucleon,current,tsep,cfg,t,corr
0,proton,A3,3,78.0,0,-3.462603e-10
1,proton,A3,3,78.0,1,8.627543e-11
2,proton,A3,3,78.0,2,2.859727e-10
3,proton,A3,3,78.0,3,4.257450e-11
4,proton,A3,3,84.0,0,5.371521e-10


In [168]:
def avg_data(arg):
    corr_avg = gv.dataset.avg_data(
        arg.pivot(index="cfg", columns="t", values="corr").values
    )
    return pd.Series(corr_avg)


group = isospin_spin_parity_avg_df.groupby(["nucleon", "current", "tsep"])
corr_df = (
    group.apply(avg_data)
    .reset_index(level=-1)
    .rename(columns={"level_3": "t", 0: "corr"})
    .reset_index()
    .set_index(["nucleon", "current", "tsep", "t"])
)
# print(corr_df)
corr_out = corr_df.to_dict(orient='tight')
# print(corr_out)
test = corr_df.loc[('proton', 'A3',3),'corr']
out = test.to_dict()
print(out)
list(out.keys())


{0: -6(42)e-12, 1: -1.1(1.0)e-10, 2: -1.03(98)e-10, 3: 5(45)e-12}


[0, 1, 2, 3]

In [163]:
for t_sink in out.keys():
    print(t_sink)
    if not isinstance(t_sink, int):
        raise TypeError("t_sink keys must be integers.")
    if nt is None:
        try:
            np.unique([len(arr) for arr in out.values()]).item()
        except ValueError as _:
            raise ValueError("Values in ydict must have same length.")

(3, 0)


TypeError: t_sink keys must be integers.

In [49]:
import fitter.corr_functions as cf 
cf.C_2pt(tag='corr',ydata=corr_df)


ValueError: Unable to coerce to Series, length must be 1: given 160

In [21]:
# h5fname = '/home/gbradley/c51_corr_analysis/tests/data/C13/C13-b_4002.ama.h5'


   
with h5py.File(h5fname, 'r') as h5f:
    # top_keys = [key for key in h5f.keys()]
    # for top_key in top_keys:
    #     group = h5f[top_key]
    #     group['10'] = group['3pt_tsep10']
    data = {}
    dsets = get_dsets(h5f)
    # print(dsets)
    for key in dsets.keys():
        # print(key)
        data[key] = h5f[key][:]
    print(data['2pt/ext_current/src5.0_snk5.0/ext_axial_A1_A1/C13.b_4002/AMA'])
    # [val for key,val in dsets.items() if key in mystring]
    # print(val[key])
    # for key in dsets.keys():
    #     if key in 
        
            
    #     dset = ifile['data']
    #     for key in dset.keys():
    #         data[key] = ifile['data'][key][:]
    # for key in ['13','14','15','16']:
    #     data[int(key)] = data.pop(key)

    # return data

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/gbradley/c51_corr_analysis/tests/a09m135_s_avg_srcs0-15.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [169]:

# import fitter.corr_functions as cf
# import fitter.fit_twopt

directory = '/home/gbradley/c51_corr_analysis/tests/data/C13/'
N_cnf = len([name for name in os.listdir(directory) if os.path.isfile(name)])

dirs = os.listdir( directory )

cnf_abbr = [files.split(".ama.h5",0) for files in dirs]

# data_file_list = os.path.realpath(dirs)
data_file_list = list()
for dirpath,_,filenames in os.walk(directory):
    for f in filenames:
        data_file_list.append(os.path.abspath(os.path.join(dirpath, f)))
file = data_file_list[0]


NUCL: nucleon
U: quark bilinear operator inserted on up-quark; D will be used for down-quark
MIXED: "mixed" type of spin projection is used
NONREL: non-relativistic proton is used
l0:  when inserting the quark bilinear oprator, the separation of the quarks of the bilinear operator is zero (local operator); you might see some l1 (quark bilinear operator separated by 1 lattice unit) data as well
g13: the gamma matrix of the quark bilinear operator is "13" in Chroma convention. Page 6 and 7 of the attached pdf shows the Chroma gamma matrix convention and its indexing; their indexing is summarized below:
 
0: scalar; I
15: pseudoscalar; g_5
1: vector;  g_x
2: vector;  g_y
4: vector;  g_z
8: vector;  g_t
14: axial;   g_x g_5
13: axial;  -g_y g_5
11: axial;   g_z g_5
7: axial;  -g_t g_5
9: tensor;  g_x g_t
10: tensor;  g_y g_t
12: tensor;  g_z g_t
3: tensor;  g_x g_y
6: tensor;  g_y g_z
5: tensor;  g_x g_z

In [170]:


string = (
    "3pt_tsep12/NUCL_D_MIXED_NONREL_l0_g0/src5.0_snk5.0/qz+0_qy+0_qx+0/C13.b_5682/AMA"
)

patterns = [
    "3pt",
    "_tsep(?P<tsep>[0-9]|[0-9]+)",  # must match `_tsep` and stores the following numbers (any length)
    "/NUCL_(?P<quark>U|D)",  # Store U or D in quark
    "_MIXED_NONREL",  # Not sure if this changes. Not stored for now
    "_l(?P<l>[0-9]+)",  # action parameters?
    "_g(?P<g>[0-15]+)",
    "/src(?P<src>[0-9\.]+)",  # Stores numbers + . to store decimals. Must escape .
    "_snk(?P<snk>[0-9\.]+)",  # Stores numbers + . to store decimals. Must escape .
    "/qz(?P<qz>[\+\-0-9]+)", 
    "_qy(?P<qy>[\+\-0-9]+)", 
    "_qx(?P<qx>[\+\-0-9]+)", 
    
]

for n in range(len(patterns)):
    pattern = "".join(patterns[:n+1])
    match = re.match(pattern, string)
    if not match:
        print(pattern)
        break

if match:
    print(match.groupdict())
# 3pt_tsep8/NUCL_U_MIXED_NONREL_l0_g9/src5.0_snk5.0/qz-3_qy+0_qx+1/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A1_A1/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A1_A1_px1_py0_pz0/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A3_P_px1_py0_pz0/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A4_A4_px3_py1_pz0/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A4_P/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A4_P_px1_py0_pz0/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_vector_T12_T12_px1_py0_pz0/C13.b_5682/AMA
# 2pt/ext_current_SP/src5.0_snk5.0/ext_vector_V2_V2_px1_py1_pz1/C13.b_5682/AMA
# 2pt/pion/src5.0_snk5.0/pion_px1_py0_pz0/C13.b_5682/AMA
# 2pt/pion_SP/src5.0_snk5.0/pion_px1_py0_pz0/C13.b_5682/AMA
# 2pt/proton/src5.0_snk5.0/proton_px1_py0_pz0/C13.b_5682/AMA
# 2pt/proton_SP/src5.0_snk5.0/proton_px1_py0_pz0/C13.b_5682/AMA

{'tsep': '12', 'quark': 'D', 'l': '0', 'g': '0', 'src': '5.0', 'snk': '5.0', 'qz': '+0', 'qy': '+0', 'qx': '+0'}


In [96]:
def parse_baryon_tag(datatag):
    datatag_split = datatag.split('/')
    corr_type     = datatag_split[0]
    tsep          = int(corr_type.split('_tsep')[1])
    buffer        =  datatag_split[1]
    channel       = buffer.split('_')[0]
    quark_ins       = buffer.split('_')[1]
    spin_proj       = buffer.split('_')[2]
    quark_sep       = buffer.split('_')[3]
    gamma           = buffer.split('_')[4] #gamma matrix of quark bilinear operator in the CHROMA convention , value accessed via dict
    src_snk_sep     = datatag_split[2]
    mom         = datatag_split[3]
    mom0       = mom.split('_')[0]
    mom1       = mom.split('_')[1]
    mom2       = mom.split('_')[2]
    momentum        = (mom0,mom1,mom2)
    config   = datatag_split[4]

    data_dict = dict()
    data_dict['corr_type']   = corr_type
    data_dict['tsep']        = tsep
    data_dict['buffer']      = buffer
    data_dict['channel']     = channel
    data_dict['quark_ins']   = quark_ins
    data_dict['spin_proj']   = spin_proj
    data_dict['quark_sep']   = quark_sep
    data_dict['gamma']       = gamma
    data_dict['src_snk_sep'] = src_snk_sep
    data_dict['mom']         = momentum
    data_dict['config']      = config
    return data_dict

   


In [97]:
parse_baryon_tag(string)

{'corr_type': '3pt_tsep12',
 'tsep': 12,
 'buffer': 'NUCL_D_MIXED_NONREL_l0_g0',
 'channel': 'NUCL',
 'quark_ins': 'D',
 'spin_proj': 'MIXED',
 'quark_sep': 'NONREL',
 'gamma': 'l0',
 'src_snk_sep': 'src5.0_snk5.0',
 'mom': ('qz+0', 'qy+0', 'qx+0'),
 'config': 'C13.b_5682'}

In [171]:
columns = ["tsep", "quark", "l", "g", "src", "snk","qz","qy","qx"]


In [178]:
data_frames = []

with h5py.File(file, "r") as h5f:
    dsets = get_dsets(h5f)
    # print(dsets)
    for key, dset in dsets.items():
        match = re.search(pattern, string)
        if match:
            info = match.groupdict()
            # print(info)
#             corr = info.pop("tsep")

            quark = info.pop("quark")
            # print(quark)
            info["quark"] = quark[0]
# #             # info["parity"] = -1 if len(nucleon_parity) == 2 else 1
            
            gamma = info.pop("g")
            if gamma in ["g1","g2","g4","g8"]:
                info["gamma"] = "vector"
            elif gamma in ["g0"]:
                info["gamma"] = "scalar"
            elif gamma in ["g5"]:
                info["gamma"] = "pseudoscalar"
            elif gamma in ["g14","g13","g11","g7"]:
                info["gamma"] = "axial"
            elif gamma in ["g14","g13","g11","g7"]:
                info["gamma"] = "axial"
            elif gamma in ["g9","g10","g12","g3","g6","g5"]:
                info["gamma"] = "tensor"

            # current_key = key.replace("g", "")
            curr_dset = h5f[key]

            cfgs = dset[:]
            corr = (
                curr_dset[()].real 
                # if info["current"] in ["V4"] else curr_dset[()].imag
            )
            # print(corr.shape[-1])
            ts = range(corr.shape[-1])
            # print(ts)
            tmp_df = (
                pd.DataFrame(index=cfgs, columns=ts, data=corr)
                .unstack()
                .reset_index()
                .rename(columns={"level_0": "tsep", "level_1": "cfg", 0: "corr"})
            )
            # data_frames = {}
            ydict = {}
            for key, val in info.items():
                tmp_df[key] = val
            data_frames.append(tmp_df.astype({"tsep": int}))
            # print(data_frames)


df = pd.concat(
    data_frames, 
    ignore_index=True, 
).reindex(columns, axis=1).sort_values(columns).reset_index(drop=True)

# df.head()
# print(df.keys())
# for 'tsep' in df.keys():
#     if not isinstance(tsep, int):
#                 raise TypeError("t_sink keys must be integers.")

[2022-08-16 20:27:20,527|lqcd correlator analysis@INFO] Locating all dsets of h5 file `/home/gbradley/c51_corr_analysis/tests/data/C13/C13-b_5682.ama.h5`


,tsep,g
tsep,NaN,NaN
g,NaN,NaN


In [46]:
import corr_functions as cf 
print(df.tsep)
ydict = {tag: val for tag, val in df.items() if isinstance(tag, int)}
ydict


0          12
1          12
2          12
3          12
4          12
           ..
3729211    12
3729212    12
3729213    12
3729214    12
3729215    12
Name: tsep, Length: 3729216, dtype: int64


{}

## statistical average ##


In [ ]:
def avg_data(arg):
    corr_avg = gvar.dataset.avg_data(
        arg.pivot(index="cfg", columns="t", values="corr").values
    )
    return pd.Series(corr_avg)


group = isospin_spin_parity_avg_df.groupby(["nucleon", "current", "tsep"])
corr_df = (
    group.apply(avg_data)
    .reset_index(level=-1)
    .rename(columns={"level_3": "t", 0: "corr"})
    .reset_index()
    .set_index(["nucleon", "current", "tsep", "t"])
)

corr_df.head()

## momentum average ##

In [ ]:
def mom_avg(h5_data,state,mom_lst,weights=False):
    '''
    perform a momentum average of a state from an open h5 file
    data file is assumed to be of shape [Nt,Nz,Ny,Nx,[re,im]]
    data_mom = h5_data[state][:,pz,py,px]
    '''
    d_lst = []
    w = []
    for mom in mom_lst:
        px,py,pz = mom['momentum']
        w.append(mom['weight'])
        #print(state)
        d_lst.append(h5_data[state][:,pz,py,px])
    d_lst = np.array(d_lst)
    w = np.array(w)
    if weights:
        for wi,we in enumerate(w):
            d_lst[wi] = we*d_lst[wi]
        d_avg = np.sum(d_lst,axis=0) / np.sum(w)
    else:
        d_avg = np.mean(d_lst,axis=0)
    return d_avg
# mom_avg('/home/gbradley/c51_corr_analysis/tests/data/C13/C13-b_5178.ama.h5', state, mom_lst)

mom_lst = []
for px in range(-2,3):
    for py in range(-2,3):
        for pz in range(-2,3):
            if px**2 + py**2 + pz**2 <= 5:
                mom_lst.append('pz'+str(pz)+'_py'+str(py)+'_px'+str(px))